<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

**Описание данных**
Набор данных находится в файле `insurance.csv`.

**Признаки:** пол, возраст и зарплата застрахованного, количество членов его семьи.

**Целевой признак:** количество страховых выплат клиенту за последние 5 лет.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

## Загрузка данных

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
display(data.head())
print(data.info())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None


In [3]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


**Выводы:**
- Общий объем данных 5000 записей
- Все столбцы имеют соответствующий тип данных
- Пропущенные значения отсутствуют
- Аномальные значения не выявлены

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Для проверки подтверждаем, что линейная регрессия в scikit-learn действительно вычисляет веса согласно указанной формуле:
- Обучить линейную регрессию на исходных данных.
- Вручную вычислить веса $w$ с помощью упомянутой формулы.
- Сравнить результаты с весами, полученными от модели scikit-learn.

In [4]:
X, y = make_regression(n_samples=100, n_features=1, noise=0.1, random_state=42)
model = LinearRegression().fit(X, y)
print("Веса из scikit-learn:", model.intercept_, model.coef_)

X_with_bias = np.c_[np.ones(X.shape[0]), X] # Расширяем матрицу X для учета смещения

# Вычисление веса вручную с помощью формулы
w_manual = np.linalg.inv(X_with_bias.T.dot(X_with_bias)).dot(X_with_bias.T).dot(y)
print("Расчетные веса:", w_manual[0], w_manual[1:])

Веса из scikit-learn: 0.011651153205267661 [41.76806101]
Расчетные веса: 0.011651153205270992 [41.76806101]


**Ответ:** b. Качество линейной регрессии не изменится при умножении признаков на обратимую матрицу


**Обоснование:** Формула для вычисления весов $w$ в линейной регрессии имеет вид: $w = (X^T X)^{-1} X^T y$

Допустим, мы умножили нашу матрицу признаков $X$ на обратимую матрицу $P$, тогда новая матрица признаков будет $XP$. $X′ =XP$

**Обучение на преобразованных данных:**
$w′ =(X′^T X′ )^{−1} X′^T y$
 
Подставим выражение для $X′$:
$w′ =(P^T X^T XP)^{−1} P^T X^T y$

Далее, если мы рассмотрим выражение для предсказания и подставим наше выражение для $w′$, мы получим:
$a′ =XP(P^T X^T XP)^{−1} P^T X^T y$

Для умножения матриц :
$(AB)^{−1}=B^{−1} A^{−1}$

Подставим в формулу обучения:
$a′ =XPP^{−1} (X^T X)^{−1} X^T y$

Поскольку $PP^{−1}$ является единичной матрицей, это упрощается до:
$a′ =X(X^T X)^{−1} X^T y$

Что ровно:
$a′ =Xw$

## Алгоритм преобразования

**Алгоритм**
- Сгенерируем случайную обратимую матрицу размером, равным количеству признаков.
- Умножим матрицу признаков на эту обратимую матрицу.
- Данные будут преобразованы, но при обучении модели на преобразованных данных качество линейной регрессии останется прежним.

**Обоснование**
Предположим, что у нас есть матрица признаков $X$ и обратимая матрица $P$. После умножения, наша новая матрица признаков будет $X′=XP$

Теперь, рассмотрим формулу обучения весов линейной регрессии:
$w=(X^TX)^{−1}X^Ty$

Если мы применяем это для преобразованных данных, то:

$w′=(X′^TX′)^{−1}X′^T y$

$w′=((XP)^T XP)^{−1}(XP)^T y$

$w′=(P^T X^T XP)^{−1}P^T X^T y$

Теперь, из свойств матриц:

$(AB)^T=B^T A^T$
$(AB)^{−1} =B^{−1} A^{−1}$

Таким образом, если $A=X^T X$ и$B=P^T P$, то:

$w′=(P^T X^T XP)^{−1}P ^T X^T y = (P^T (X^T X)P)^{−1}P ^T X^T y$

Так как умножение матрицы на её обратную дает единичную матрицу, и умножение на единичную матрицу не влияет на результат, качество линейной регрессии не изменится

Раскроем скобки, с учетом свойства $(AB)^{−1} =B^{−1} A^{−1}$:

$w′ =P^{−1} (X^T X) ^{−1} P^{−T} P^T X^T y$

$P^{−T} P^T$ дают единичную матрицу, поскольку произведение матрицы и её обратной равно единичной матрице:

$w′ =P^{−1} (X^T X)^{−1} X^T y$

Известно что: 
$w=(X^T X)^{−1} X^T y$

Таким образом, получаем:
$w′ =P^{−1} w$

Предсказания для оригинальных данных:
$a=Xw$

Для преобразованных данных:
$a′ =X′ w′ =XPP^{−1} w=Xw=a$

Таким образом,  𝑎′
  =  𝑎
  и предсказания не изменятся после умножения на обратимую матрицу. Качество линейной регрессии останется неизменным.

## Проверка алгоритма

1. Создадим и обучим модель линейной регрессии на оригинальных данных.
2. Сгенерируем обратимую матрицу и умножим на неё наши данные.
3. Обучим новую модель линейной регрессии на преобразованных данных.
4. Сравним метрики R2 обеих моделей на оригинальных и преобразованных данных.

In [5]:
X, y = make_regression(n_samples=1000, n_features=5, noise=0.1)

# Обучение модели на оригинальных данных
model_original = LinearRegression().fit(X, y)
predictions_original = model_original.predict(X)
r2_original = r2_score(y, predictions_original)
 # Генерация обратимой матрицы
P = np.random.rand(X.shape[1], X.shape[1])
while np.linalg.det(P) == 0:
    P = np.random.rand(X.shape[1], X.shape[1])

X_transformed = X.dot(P)

model_transformed = LinearRegression().fit(X_transformed, y)
predictions_transformed = model_transformed.predict(X_transformed)
r2_transformed = r2_score(y, predictions_transformed)

print(f"R2 оригинальной модели: {r2_original}")
print(f"R2 модели на преобразованных данных: {r2_transformed}")

R2 оригинальной модели: 0.9999994020795202
R2 модели на преобразованных данных: 0.9999994020795202


In [6]:
X = data.drop('Страховые выплаты', axis=1 )
y = data['Страховые выплаты']

model_original = LinearRegression().fit(X, y)
predictions_original = model_original.predict(X)
r2_original = r2_score(y, predictions_original)

# Генерация обратимой матрицы
P = np.random.rand(X.shape[1], X.shape[1])
while np.linalg.det(P) == 0:
    P = np.random.rand(X.shape[1], X.shape[1])

X_transformed = X.dot(P)

model_transformed = LinearRegression().fit(X_transformed, y)
predictions_transformed = model_transformed.predict(X_transformed)
r2_transformed = r2_score(y, predictions_transformed)

print(f"R2 оригинальной модели: {r2_original}")
print(f"R2 модели на преобразованных данных: {r2_transformed}")

R2 оригинальной модели: 0.4249455028666801
R2 модели на преобразованных данных: 0.4249455028659693


**Общие выводы:**

R2 оригинальной модели и модели на преобразованных данных совпадает. Это подтверждает что преобразование признаков с использованием обратимой матрицы не влияет на качество линейной регрессии.

Таким образом, можно безопасно преобразовывать данные клиентов с целью их шифрования, сохраняя при этом качество модели. 

Для достижения этой цели использовали алгоритм, который многократно умножает данные на случайную обратимую матрицу. 

Теоретически обосновали, что это преобразование не изменяет качества линейной регрессии.